In [ ]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from koopmanlib.dictionary import PsiNN
from koopmanlib.solver import KoopmanDLSolver
from matplotlib import pyplot as plt

In [ ]:
dim = 3  
hidden_dim = 20
input_dim = 0
n_feature = 17
num_epochs = 100
batch_size = 512
n_train = 10000
n_valid = 1000
n_test = 1000
learning_rate = 1e-3

In [ ]:
X_train = pd.read_csv('Non_X_train.csv', header=None).values
X_valid = pd.read_csv('Non_X_valid.csv', header=None).values
X_test = pd.read_csv('Non_X_test.csv', header=None).values
U_train = pd.read_csv('Non_U_train.csv', header=None).values
U_valid = pd.read_csv('Non_U_valid.csv', header=None).values
U_test = pd.read_csv('Non_U_test.csv', header=None).values

length = X_train.shape[1] // n_train
HX_train = []
HU_train = []
for i in range(n_train):
    HX_train.append(X_train[:, i*length:(i+1)*length])
    HU_train.append(U_train[:, i*length:(i+1)*length])
HX_train = np.stack([HX_train[idx].T for idx in range(n_train)], axis=0)
HU_train = np.stack([HU_train[idx].T for idx in range(n_train)], axis=0)
HX_valid = []
HU_valid = []
for i in range(n_valid):
    HX_valid.append(X_valid[:, i*length:(i+1)*length])
    HU_valid.append(U_valid[:, i*length:(i+1)*length])
HX_valid = np.stack([HX_valid[idx].T for idx in range(n_valid)], axis=0)
HU_valid = np.stack([HU_valid[idx].T for idx in range(n_valid)], axis=0)

H_train = np.concatenate([HX_train, HU_train], axis=-1)
H_valid = np.concatenate([HX_valid, HU_valid], axis=-1)

In [ ]:
data_train = [H_train[:, 0:-1,:].reshape(-1, dim), H_train[:, 1:,:].reshape(-1, dim)]
data_valid = [H_valid[:, 0:-1,:].reshape(-1, dim), H_valid[:, 1:,:].reshape(-1, dim)]
basis_function = PsiNN(layer_sizes=[hidden_dim, hidden_dim, hidden_dim], n_psi_train=dim+n_feature)
solver = KoopmanDLSolver(dic=basis_function,
                             target_dim=dim,
                             reg=0.0)
solver.build(data_train=data_train,
                 data_valid=data_valid,
                 epochs=num_epochs,
                 batch_size=batch_size,
                 lr=learning_rate,
                 log_interval=1,
                 lr_decay_factor=0.92)
    
# df.to_csv("noise.csv", index=False)

In [ ]:
length = X_test.shape[1] // n_test
HX_test = []
HU_test = []
for i in range(n_test):
    HX_test.append(X_test[:, i*length:(i+1)*length])
    HU_test.append(U_test[:, i*length:(i+1)*length])
HX_test = np.stack([HX_test[idx].T for idx in range(n_test)], axis=0)
HU_test = np.stack([HU_test[idx].T for idx in range(n_test)], axis=0)

H_test = np.concatenate([HX_test, HU_test], axis=-1)

In [ ]:
x_pred_list = []
for i in range(n_test):
    x0_test = H_test[i, :1, :]
    x_pred_list.append(solver.predict(x0_test, 60)[:, :2])
X_pred = np.stack(x_pred_list, axis=0)

In [ ]:
np.mean((X_pred - HX_test[:, 1:, :])**2)

In [ ]:
np.save('mse_edmddl.npy', np.mean((X_pred - HX_test[:, 1:, :])**2, axis=(0, 2)))

In [ ]:
mse = np.load('mse_edmddl.npy')